## Insurer Level Dataset Integration

This Google Colab notebook aims to integrate the three main datasets and get an overview of insurer level data. It includes the combination of three datasets which are Marktanteile je Kasse, Morbidity Region and Zusatzbeitrag_je Kasse je Quartal. The files Marktanteile je Kasse and Morbidity Region have already been merged manually and have been renamed to Insurer_Level_Data in this notebook. We focus on combining that with the Zusatzbeitrag_je Kasse je Quartal dataset.

In [29]:
import pandas as pd
from google.colab import files

In [30]:
quarterly_file = "Zusatzbeitrag_je Kasse je Quartal.xlsx"
annual_file = "Insurer_Level_Data.xlsx"

quarterly_df = pd.read_excel(quarterly_file)
annual_df = pd.read_excel(annual_file)

Below are certain preprocessing steps taken to ensure a clean merge between the two datasets. There are some discrepancies in the naming conventions of the Krankenkasse which have been identified and resolved here.

In [31]:
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("AOK NORDWEST", "AOK NordWest")
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("BKK Dürkopp Adler", "BKK DürkoppAdler")
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("BKK Melitta-HMR", "BKK HMR")
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("BKK der MTU Friedrichshafen", "BKK MTU")
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("BKK Rieker.Ricosta.Weisser", "BKK Rieker.RICOSTA.Weisser")
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("Hanseatische Krankenkasse (HEK)", "HEK")
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("IKK - Die Innovationskasse", "IKK – Die Innovationskasse")
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("BKK Metzinger", "Metzinger BKK")
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("Techniker Krankenkasse (TK)", "Techniker-Krankenkasse (TK)")
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("actimonda krankenkasse", "actimonda Krankenkasse")
annual_df["Krankenkasse"] = annual_df["Krankenkasse"].replace("vivida bkk", "vivida BKK")

Below we get an aggregated average of the values of Mitglieder, Versicherte and Zusatzbeitrag for the quarterly dataset and then merge them into a single row with the annual level data. Information within the years 2013 - 2015 are dropped as the annual level dataset only has data up until 2016. Finally, we remove all Krankenkasse's which do not have a corresponding record in the other dataset. This is done to avoid skewing of results.

In [32]:
# Aggregate by Krankenkasse and Jahr
quarterly_agg = quarterly_df.groupby(["Krankenkasse", "Jahr"]).agg({
    "Mitglieder": "mean",
    "Versicherte": "mean",
    "Zusatzbeitrag": "mean"
}).reset_index()

# Merge aggregated quarterly data with the annual data
combined_df = pd.merge(quarterly_agg, annual_df, on=["Krankenkasse", "Jahr"], how="left")

# Drop rows where 'Jahr' is 2013, 2014, or 2015
combined_df = combined_df[~combined_df["Jahr"].isin([2013, 2014, 2015])]

combined_df = combined_df[combined_df["Krankenkasse"] != "SVLFG"]
combined_df = combined_df[combined_df["Krankenkasse"] != "BKK Grillo-Werke"]
combined_df = combined_df[combined_df["Krankenkasse"] != "BKK Melitta-HMR"]
combined_df = combined_df[combined_df["Krankenkasse"] != "BKK advita"]
combined_df = combined_df[combined_df["Krankenkasse"] != "BKK Beiersdorf AG"]
combined_df = combined_df[combined_df["Krankenkasse"] != "BKK Braun-Gillette"]

Further preprocessing is done below. Firstly we remove rows with region as "betriebsbezogen" as they are accessible to employees only within their respective company. We then replace any '-' values with '0's for easier analysis

In [35]:
# Remove insurances of type "betriebsbezogen" -> As they are only accessible to employees within a company and not the general public
combined_df = combined_df[combined_df["Regionalität"] != "betriebsbezogen"]

# Preprocess all '-' values with 0
cols_to_clean = [
    "Mitglieder",
    "Versicherte",
    "Zusatzbeitrag",
    "Marktanteil Versicherte",
    "Marktanteil Mitglieder",
    "Risikofaktor",
    "RF-Entwicklung im Vgl zum Vorjahr"
]

# Apply .loc to replace '-' with 0 across these columns
combined_df.loc[:, cols_to_clean] = combined_df.loc[:, cols_to_clean].replace('-', 0)

In [33]:
combined_df.head()

,Krankenkasse,Jahr,Mitglieder,Versicherte,Zusatzbeitrag,Marktanteil Versicherte,Marktanteil Mitglieder,Risikofaktor,RF-Entwicklung im Vgl zum Vorjahr,Regionalität,Regionale Verteilung
3,AOK Baden-Württemberg,2016,3125675.50,4101129.75,1.0,0.058070,0.057326,1.001319,-0.007768,Regional,Baden-Württemberg
4,AOK Baden-Württemberg,2017,3236896.50,4244203.00,1.0,0.059496,0.058483,0.985325,-0.015973,Regional,Baden-Württemberg
5,AOK Baden-Württemberg,2018,3349949.75,4384184.00,1.0,0.060912,0.059854,0.967854,-0.017731,Regional,Baden-Württemberg
6,AOK Baden-Württemberg,2019,3393316.50,4448225.50,0.9,0.061500,0.060100,0.972076,0.004363,Regional,Baden-Württemberg
7,AOK Baden-Württemberg,2020,3439196.75,4512117.50,0.9,0.062151,0.060701,0.966822,-0.005405,Regional,Baden-Württemberg


Finally, we download the new dataset in Excel format. This can be used for EDA and statistical analysis for the insurance companies.

In [36]:
# First, install openpyxl (required for writing Excel files)
!pip install openpyxl

# Export to Excel file
output_excel = "Final_Insurer_Level_Dataset.xlsx"
combined_df.to_excel(output_excel, index=False, engine='openpyxl')

files.download(output_excel)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>